In [ ]:
import os

current_path = os.getcwd()
dataset_path = os.path.join(current_path,"..\\dataset")
    
novelSet_path = os.path.join(dataset_path, "visual-novels\\Parsed")
save_novelJson_path = os.path.join(dataset_path, "visual-novels-txt-v4")

print("novelSet_path: ", novelSet_path)

novel_list = os.listdir(novelSet_path)

In [33]:
def create_folder(_path):
    if not os.path.exists(_path):
        os.makedirs(_path, exist_ok=True)

In [34]:
create_folder(dataset_path)
create_folder(save_novelJson_path)

In [35]:
class dict(object):
    def __init__(self, name):
        self._name = name

    def __str__(self):
        return self._name

    def __repr__(self):
        return "'"+str(self._name)+"'"

class Qmark:
    def __init__(self):
        self.sig=False
        self.cnt=0
        self.idx=0
        self.json_dict=[]
        self.novel_title=""
    
    def add_cnt(self):
        self.cnt+=1
    def add_idx(self):
        self.idx+=1
        
    def get_cnt(self):
        return self.cnt
    def get_idx(self):
        return self.idx

    def reset_cnt(self):
        self.cnt=0
    def reset_idx(self):
        self.idx=0

    def set_sig_true(self):
        self.sig=True
    def set_sig_false(self):
        self.sig=False
    def get_sig(self):
        return self.sig
    
    def reset_json_dict(self):
        self.json_dict.clear()
    def add_json_dict(self, name, content):
        self.json_dict.append({name:content})
    def get_json_dict(self):
        return self.json_dict
    def get_json_dict_len(self):
        return len(self.json_dict)
    
    def set_novel_title(self, title):
        self.novel_title=title
    def get_novel_title(self):
        return self.novel_title

In [36]:
def qeustion_marker(name, content, qmark):
    if content == "":
        return
    try:
        content.replace("<USER>", "you")
        if qmark.get_cnt()>3:
            if qmark.get_sig() & ('?' in content):
                qmark.set_sig_false()
                
            if qmark.get_sig():
                qmark.set_sig_false()
                qmark.reset_cnt()
                qmark.add_json_dict('question', "===============================")
                qmark.add_json_dict(name,content)
                qmark.add_json_dict('answer', "===============================")
                return
            
        if '?' in content[-1]:
            qmark.add_json_dict(name, content)
            qmark.add_cnt()
            qmark.set_sig_true()
        else:
            qmark.add_json_dict(name, content)
            qmark.set_sig_false()
            qmark.add_cnt()
    except:
        print("qeustion_marker error: ", content)

In [37]:
class TrainData():
    def __init__(self):
        self.trainSet=[]
        self.input_novel=[]
        self.answer=[]
        self.output_novel=[]
        self.train={}
        self.q=False
        self.a=False
        
    def set_q(self, boolean):
        self.q=boolean
    def set_a(self, boolean):
        self.a=boolean
        
    def get_q(self):
        return self.q
    def get_a(self):
        return self.a
    
    def add_value(self, tag, name, content):
        format = self.input_format(name, content)
        if tag =="input_novel":
            self.input_novel.append(format)
        elif tag =="answer":
            self.answer.append(format)
        elif tag =="output_novel":
            self.output_novel.append(format)
        elif tag == "train":
            self.train[name] = content.copy()
        elif tag == "trainSet":
            self.trainSet.append(content.copy())
        else:
            print("No such key exist.")
            
    def input_format(self, name, content):
        return f"{name}:{content}"
    
    def reset_value(self, name):
        if name == "input_novel":
            self.input_novel.clear()
        elif name == "answer":
            self.answer.clear()
        elif name == "output_novel":
            self.output_novel.clear()
        elif name == "train":
            self.train.clear()
            
    def get_value(self, name):
        if name =="input_novel":
            return self.input_novel
        elif name =="answer":
            return self.answer
        elif name =="output_novel":
            return self.output_novel
        elif name =="train":
            return self.train
        elif name =="trainSet":
            return self.trainSet
        else:
            print("No such key exist.")
            
    def change_trainSet(self):
        self.input_novel = self.output_novel.copy()
        self.reset_value('answer')
        self.reset_value('output_novel')
        self.reset_value('train')

In [38]:
def conversion_to_trainData(qmark):
    def save_value():
        train_data.add_value('train', 'input_novel', train_data.get_value('input_novel'))
        train_data.add_value('train', 'answer', train_data.get_value('answer'))
        train_data.add_value('train', 'output_novel', train_data.get_value('output_novel'))
        train_data.add_value('trainSet', 'train', train_data.get_value('train'))
    
    qmark_data = qmark.get_json_dict()
    q=0
    train_data = TrainData()
    for qmark_data in qmark.get_json_dict():
        for name, value in qmark_data.items():
            if(str(name) == "question"):
                train_data.set_q(True)
                q+=1
                if(q!=1):
                    save_value()
                    
                    train_data.change_trainSet()
                    train_data.set_a(False)
                continue
            if(str(name) == "answer"):
                train_data.set_a(True)
                continue
                    
            if not train_data.get_q() and not train_data.get_a():
                train_data.add_value('input_novel', name, value)
            elif train_data.get_q() and not train_data.get_a():
                train_data.add_value('answer', name, value)
            elif train_data.get_q() and train_data.get_a():
                train_data.add_value('output_novel', name, value)
            else:
                print("Error: No matching question or answer found.")
    
    if train_data.get_value('input_novel'):
        save_value()
    return train_data.get_value('trainSet')

In [39]:

case_1 = ['A Clockwork Ley-Line - The Borderline of Dusk']
    #"TO:~ 로 시작하는 편지 존재
case_2 = ['AIR']
    #나레이션 태그 없음, *감탄사 존재
case_3 = ['Amatarasu Riddle Star', 'Maitetsu']
    #**~** 안의 내용 제거
case_4 = ['Aokana - Four Rhythms Across the Blue']
    #나레이션 태그 없음, <i></i> 형태의 강조 태그 존재
case_5 = ['Blackberry Honey']
    #대화문장 뒤 with dissolve존재, 대화 문장 내부에 {}, 언더바 존재
case_6 = ['Chasing Tails A Promise In the Snow']
    #나레이션 태그 없음
case_8 = ['Cross Worlds','Sisterly Bliss (Tsui Yuri ~Okaa-san ni wa Naisho da yo~)', 'The Eden of Grisaia', 'The Fruit of Grisaia', 'The Labyrinth of Grisaia']
    #Narrator, Narraition라는 name 존재
case_9 = ['CrossChannel']
    #나레이션 태그 없음, 『』존재, 나레이션 태그가 효과음과 동작을 나타냄.
case_10 = ['Dies Irae - Acta Est Fabula']
    #{i} ~{/i} 안의 내용 제거, {}안의 내용 제거
case_12 = ['DRACU-RIOT!']
    #%32, %36, %0 존재
case_13 = ['Driven Affairs']
    #-GASP 존재
case_14 = ['Everlasting Summer']
    #<>안의 내용 제거
case_15 = ['Fxxx Me Royally!! Horny Magical Princess (Himetai)']
    #<I> ~ </i> 안의 내용 제거, _ 제거
case_17 = ['Libra of the Vampire Princess (Kyuuketsuki no Libra)']
    #대화 안에 * 존재
case_18 = ['Muv Luv Extra' '*Muv Luv Unlimited']
    #문장안에 **, *──* 존재하면 문장 제거
case_19 = ['Rewrite']
    #나레이션 태그 없음, *snip*과 *chink* 존재
case_20 = ['Waifu Hub']
    #*( ~ )* 안의 내용 제거
case_21 = ['Wolf Tails']
    #{i} ~ {/i} 안의 내용 제거, [player_name] you로 변경, *()* 눈 you 태그로 변경, You를 you로 변경
case_22 = ['Shikkoku no Sharnoth ~What a beautiful tomorrow~', 'Euphoria', 'Girls Frontline (Extras)', 'Girls Frontline', 'MonsterGirlQuest']
    #제외
case_23 = ['Dawn Of Malice', 'Love At First Sight', 'Academy34', 'Dawn Of Malice', 'Doki Doki Literature Club', 'Prince Of Suburbia', 'REFLEXIA Prototype ver', 'Sugar Style', 'Waifu Hub']
    #YOU, you, user, <USER>를 you로 변경+
    

In [40]:
def exception(qmark, novel_content):
    title = qmark.get_novel_title()
    if title in case_1:
        case_1_func(qmark, novel_content)
    elif title in case_2:
        case_2_func(qmark, novel_content)
    elif title in case_3:
        case_3_func(qmark, novel_content)
    elif title in case_4:
        case_4_func(qmark, novel_content)
    elif title in case_5:
        case_5_func(qmark, novel_content)
    elif title in case_6:
        case_6_func(qmark, novel_content)
    elif title in case_8:
        case_8_func(qmark, novel_content)
    elif title in case_9:
        case_9_func(qmark, novel_content)
    elif title in case_10:
        case_10_func(qmark, novel_content)
    elif title in case_12:
        case_12_func(qmark, novel_content)
    elif title in case_14:
        case_14_func(qmark, novel_content)
    elif title in case_15:
        case_15_func(qmark, novel_content)
    elif title in case_17:
        case_17_func(qmark, novel_content)
    elif title in case_18:
        case_18_func(qmark, novel_content)
    elif title in case_19:
        case_19_func(qmark, novel_content)
    elif title in case_20:
        case_20_func(qmark, novel_content)
    elif title in case_21:
        case_21_func(qmark, novel_content)
    elif title in case_22:
        case_22_func(qmark, novel_content)
    elif title in case_23:
        case_23_func(qmark, novel_content)
    else:
        other_case(qmark, novel_content)

In [41]:
import json

def custom_serializer(obj):
    if isinstance(obj, dict):
        return str(obj)  # 클래스의 __str__ 메서드를 사용하여 문자열로 변환
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

def save_novel(qmark):
    qmark.add_idx()
    train_set = conversion_to_trainData(qmark)
    save_to_json(qmark.get_idx(), qmark.get_novel_title(), train_set)
    qmark.reset_json_dict()

def save_to_json(idx, novel_title, result):
    idx = str(idx)
    file_name = novel_title+"_"+str(idx)+".json"
    save_json_path = os.path.join(save_novelJson_path, novel_title, file_name)
    
    with open(save_json_path, 'w', encoding='utf-8') as f:
        json.dump(
            result, 
            f, 
            default=custom_serializer, 
            indent=4, 
            ensure_ascii = False
            )

In [42]:
import re
        
def save_logic(qmark, sentence):
    if sentence == "":
        return
    try:
        if sentence[0] == "*":
            qeustion_marker('narration', sentence.strip(" \"\'*"), qmark)
        elif sentence[0] == "=":
            save_novel(qmark)
        else:
            name, content = sentence.split(":", 1)
            qeustion_marker(name.strip(), content.strip(" \"\'*"), qmark)
    except:
        qeustion_marker('narration', sentence.strip(" \"\'*"), qmark)

def del_smallBracket(sentence): # ( ) 소괄호
    return re.sub(r"\(.*\)", "", sentence)
def del_middleBracket(sentence): # [ ] 중괄호
    return re.sub(r"[.*]", "", sentence)
def del_lenrticularBracket(sentence): # 【 】 렌즈 괄호
    return re.sub(r"【.*】", "", sentence) 
def del_bigBracket(sentence): # { } 대괄호
    return re.sub(r"{.*}", "", sentence)
def del_cornerBracket(sentence): # 『 』 겹낫표 
    return re.sub(r"『.*』", "", sentence)
def del_oneStar(sentence): # * 별표 한개
    return re.sub(r"\*.*\*", "", sentence)
def del_twoStar(sentence): # ** 별표 두개
    return re.sub(r"\*\*.*\*\*", "", sentence)
def del_specialSeparator(sentence): # <i> </i> 구분자
    pattern = r"<i>.*</i>|<i>.*<i>|<i>.*<I>|<i>.*</I>|<I>.*</i>"
    return re.sub(pattern, "", sentence)
def del_angleBrackets(sentence): # < > 꺽쇠괄호
    return re.sub(r"<.*>", "", sentence)

def case_1_func(qmark, novel_contents): # "TO:~ 로 시작하는 편지 존재
    for sentence in novel_contents:
        if "To:" in sentence:
            qeustion_marker('narration', sentence.strip(" \"\'\*"), qmark)
        else: 
            save_logic(qmark, sentence)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)

    
def case_2_func(qmark, novel_contents): # 나레이션 태그 없음, * ~ * 안의 내용 제거
    for sentence in novel_contents:
        result = del_oneStar(sentence)
        
        save_logic(qmark, result)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_3_func(qmark, novel_contents): # 나레이션 태그 안에*~* 안의 내용 제거
    for sentence in novel_contents:
        if sentence[0] == "*":
            result = del_twoStar(sentence[1:-1])
            save_logic(qmark, result)
        else:    
            save_logic(qmark, sentence)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
    
def case_4_func(qmark, novel_contents): # 나레이션 태그 없음, <i></i>, <i><i>, <i><I>,<i></I> 형태의 강조 태그 존재
    for sentence in novel_contents:
        
        result = del_specialSeparator(sentence)
                
        save_logic(qmark, result)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_5_func(qmark, novel_contents): # 대화문장 뒤 with dissolve존재, 대화 문장 내부에 {}, 언더바 존재
    for sentence in novel_contents:
        result = sentence.replace("with dissolve", "")
        result = result.replace("_", "")
        result = del_bigBracket(result)
        
        save_logic(qmark, result)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_6_func(qmark, novel_contents): # {}, [] 존재
    for sentence in novel_contents:
        result = del_middleBracket(sentence)
        result = del_bigBracket(result)
        
        save_logic(qmark, result)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_7_func(qmark, novel_contents): # 나레이션 태그 없음
    for sentence in novel_contents:
        save_logic(qmark, sentence)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_8_func(qmark, novel_contents): # Narrator, Narration라는 name 존재
    for sentence in novel_contents:
        if 'Narration' in sentence or 'Narrator' in sentence:
            name, content = sentence.split(':',1)
            qeustion_marker('narration', content.strip(" \"\'\*"), qmark)
        else:
            save_logic(qmark, sentence)
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_9_func(qmark, novel_contents): # 나레이션 태그 없음, 『』존재, 나레이션 태그가 효과음과 동작을 나타냄.
    for sentence in novel_contents:
        result = del_cornerBracket(sentence)
        
        save_logic(qmark, result)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_10_func(qmark, novel_contents): # [] 안에 존재하는 내용 제거. ( )제거
    for sentence in novel_contents:
        result = sentence.replace("(", "").replace(")", "")
        result = del_middleBracket(result)
        save_logic(qmark, result)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_11_func(qmark, novel_contents): # {i} ~{/i} 안의 내용 제거, {}안의 내용 제거
    for sentence in novel_contents:
        result = del_specialSeparator(sentence)
        result = del_bigBracket(result)
        
        save_logic(qmark, result)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_12_func(qmark, novel_contents): # %32, %36, %0 존재, 나레이션 태그 사이에 * ~ * 존재(제거)
    for sentence in novel_contents:
        pattern = r"%32|%36|%0"
        result = re.sub(pattern, "", sentence)
        result = del_oneStar(result)
        
        save_logic(qmark, result)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_14_func(qmark, novel_contents): # <>안의 내용 제거
    for sentence in novel_contents:
        result = del_angleBrackets(sentence)
        
        save_logic(qmark, result)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_15_func(qmark, novel_contents): # []()【】 제거
    for sentence in novel_contents:
        result = del_smallBracket(sentence)
        result = del_middleBracket(result)
        result = del_lenrticularBracket(result)
        
        save_logic(qmark, result)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_16_func(qmark, novel_contents): # <I> ~ </i> 안의 내용 제거, _ 제거
    for sentence in novel_contents:
        result = del_specialSeparator(sentence)
        result = result.replace("_", "")
        save_logic(qmark, sentence)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_17_func(qmark, novel_contents): # 대화 안에 * 존재
    for sentence in novel_contents:
        if sentence[0] != "*":
            result = del_oneStar(sentence)
            save_logic(qmark, result)    
        else:
            save_logic(qmark, sentence)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_18_func(qmark, novel_contents): # 문장안에 * ~ * 제거, ─* 존재하면 문장 제거
    for sentence in novel_contents:
        if "─*" in sentence:
            continue
        if sentence[0] != "*":
            result = del_oneStar(sentence)
            save_logic(qmark, result)
        else:
            save_logic(qmark, sentence)
        
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_19_func(qmark, novel_contents): # 나레이션 태그 없음, *snip*과 *chink* 존재
    for sentence in novel_contents:
        if "*snip*" in sentence or "*chink*" in sentence:
            continue
        save_logic(qmark, sentence)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_20_func(qmark, novel_contents): # *( ~ )* 안의 내용 제거
    for sentence in novel_contents:
        result = re.sub(r"\*\(.*\)\*", "", sentence)
        save_logic(qmark, result)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_21_func(qmark, novel_contents): # {i} ~ {/i} 안의 내용 제거, [player_name] you로 변경, *()* 눈 user 태그로 변경, You를 you로 변경
    for sentence in novel_contents:
        result = del_specialSeparator(sentence)
        result = result.replace("[player_name]", "you")
        result = result.replace("You", "you")
        if "*(" in sentence:
            result = result.replace("*(", "").replace(")*", "")
            qeustion_marker("user", result.strip(" \"\'"), qmark)
        else:
            save_logic(qmark, result)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)
        
def case_22_func(qmark, novel_contents):
    return
def case_23_func(qmark, novel_contents):
    for sentence in novel_contents:
        if 'you:' in sentence[:5].strip().lower() or '<USER>' in sentence[:5].strip():
            name, content = sentence.split(':',1)
            qeustion_marker('user', content.strip(" \"\'\*"), qmark)
            continue
        save_logic(qmark, sentence)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)

def other_case(qmark, novel_contents):
    for sentence in novel_contents:
        result = del_smallBracket(sentence)
        result = del_middleBracket(result)
        result = del_lenrticularBracket(result)
        result = del_bigBracket(result)
        
        save_logic(qmark, result)
    
    if(qmark.get_json_dict_len()!=0):
        save_novel(qmark)

<>:40: SyntaxWarning: invalid escape sequence '\*'
<>:110: SyntaxWarning: invalid escape sequence '\*'
<>:245: SyntaxWarning: invalid escape sequence '\*'
<>:40: SyntaxWarning: invalid escape sequence '\*'
<>:110: SyntaxWarning: invalid escape sequence '\*'
<>:245: SyntaxWarning: invalid escape sequence '\*'
C:\Users\kimbo\AppData\Local\Temp\ipykernel_26936\3045286000.py:40: SyntaxWarning: invalid escape sequence '\*'
  qeustion_marker('narration', sentence.strip(" \"\'\*"), gmark)
C:\Users\kimbo\AppData\Local\Temp\ipykernel_26936\3045286000.py:110: SyntaxWarning: invalid escape sequence '\*'
  qeustion_marker('narration', content.strip(" \"\'\*"), gmark)
C:\Users\kimbo\AppData\Local\Temp\ipykernel_26936\3045286000.py:245: SyntaxWarning: invalid escape sequence '\*'
  qeustion_marker('user', content.strip(" \"\'\*"), gmark)


In [43]:
def novel_convert_to_json(novel_title, novel_contents):
    qmark = Qmark()
    
    novel_title = novel_title.strip('.txt')
    save_path = os.path.join(save_novelJson_path, novel_title)
    create_folder(save_path)
    qmark.set_novel_title(novel_title)
    
    exception(qmark, novel_contents)

In [ ]:
for novel_name in novel_list:
    open_file = os.path.join(novelSet_path, novel_name)
    f = open(open_file, 'r', encoding='utf-8')
    novel_content = []
    while True:
        line = f.readline()
        if not line: 
            break
        novel_content.append(line.strip())
    f.close()
    novel_convert_to_json(novel_name, novel_content)
    print(f'complete novel : {novel_name}')

In [ ]:
# test code
novel_name = 'Fxxx Me Royally!! Horny Magical Princess (Himetai).txt'

open_file = os.path.join(novelSet_path, novel_name)
f = open(open_file, 'r', encoding='utf-8')
novel_content = []
while True:
    line = f.readline()
    if not line: 
        break
    novel_content.append(line.strip())
f.close()
novel_convert_to_json(novel_name, novel_content)
print(f'complete novel : {novel_name}')